Import Needed Modules

In [ ]:
!pip install tensorflow-docs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 kB 8.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import random
import ssl
import cv2
import numpy as np
import imageio
from IPython import display
from urllib import request
import re
import tempfile
import pandas as pd
from keras import backend as K
import sys
import csv
import os
import cv2
import math
import datetime as dt
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from google.colab import drive
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install git+https://github.com/tensorflow/docs
import collections

import os
import cv2
import numpy as np

import tensorflow as tf

import imageio
from IPython import display
from urllib import request
from tensorflow_docs.vis import embed # After installation, this line should work correctly.

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-rs_dmuk1
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-rs_dmuk1
  Resolved https://github.com/tensorflow/docs to commit d8930cafc3defe537af1326dbd651d8a9801f83f
  Preparing metadata (setup.py) ... done


Get Data

In [ ]:
import zipfile

def list_files_from_zip(zip_path):
    files = []
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for zip_info in zip_ref.infolist():
            files.append(zip_info.filename)
    return files

URL = "/content/drive/MyDrive/Shop DataSet.zip"
files = list_files_from_zip(URL)

files = [f for f in files if f.endswith('.mp4')]

files[:10]


['Shop DataSet/non shop lifters/shop_lifter_n_0.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_0_1.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_1.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_1_1.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_10.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_10_1.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_100.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_100_1.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_101.mp4',
 'Shop DataSet/non shop lifters/shop_lifter_n_101_1.mp4']

In [ ]:
import os

def get_class(fname):
    split_path = os.path.normpath(fname).split(os.sep)
    class_name = split_path[-2]

    return class_name

In [ ]:
import collections
NUM_CLASSES = 2
def get_files_per_class(files):

  files_for_class = collections.defaultdict(list)
  for fname in files:
    class_name = get_class(fname)
    files_for_class[class_name].append(fname)
  return files_for_class


files_for_class = get_files_per_class(files)
classes = list(files_for_class.keys())

In [ ]:
print('Num classes:', len(classes))
print('Num videos for class[0]:', len(files_for_class[classes[0]]))
print('Num videos for class[1]:', len(files_for_class[classes[1]]))

Num classes: 2
Num videos for class[0]: 531
Num videos for class[1]: 324


In [ ]:
print(classes)

['non shop lifters', 'shop lifters']


In [ ]:
files_for_class[classes[1]][:5], files_for_class[classes[0]][:5]

(['Shop DataSet/shop lifters/shop_lifter_0.mp4',
  'Shop DataSet/shop lifters/shop_lifter_1.mp4',
  'Shop DataSet/shop lifters/shop_lifter_10.mp4',
  'Shop DataSet/shop lifters/shop_lifter_100.mp4',
  'Shop DataSet/shop lifters/shop_lifter_101.mp4'],
 ['Shop DataSet/non shop lifters/shop_lifter_n_0.mp4',
  'Shop DataSet/non shop lifters/shop_lifter_n_0_1.mp4',
  'Shop DataSet/non shop lifters/shop_lifter_n_1.mp4',
  'Shop DataSet/non shop lifters/shop_lifter_n_1_1.mp4',
  'Shop DataSet/non shop lifters/shop_lifter_n_10.mp4'])

In [ ]:
import random

def select_subset_of_classes(files_for_class, classes, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):

    files_split = dict()

    for class_name in classes:
        class_files = files_for_class[class_name]
        num_files = len(class_files)

        # Shuffle files to ensure randomness
        random.shuffle(class_files)

        # Determine the number of files for each split
        num_train = int(train_ratio * num_files)
        num_val = int(val_ratio * num_files)
        num_test = num_files - num_train - num_val  # The remaining files go to test

        # Split the files into train, val, and test sets
        train_files = class_files[:num_train]
        val_files = class_files[num_train:num_train + num_val]
        test_files = class_files[num_train + num_val:]

        # Store the subsets for the current class
        files_split[class_name] = {
            'train': train_files,
            'val': val_files,
            'test': test_files
        }

    return files_split


In [ ]:
files_split = select_subset_of_classes(files_for_class, classes[:NUM_CLASSES])

print(len(files_split['non shop lifters']['train']))
print(len(files_split['non shop lifters']['val']))
print(len(files_split['non shop lifters']['test']))
print(len(files_split['shop lifters']['train']))
print(len(files_split['shop lifters']['val']))
print(len(files_split['shop lifters']['test']))

424
53
54
259
32
33


In [ ]:
len(files_split['non shop lifters']['train']) + len(files_split['non shop lifters']['val']) + len(files_split['non shop lifters']['test']) + len(files_split['shop lifters']['train']) + len(files_split['shop lifters']['val']) + len(files_split['shop lifters']['test'])

855

In [ ]:
import pandas as pd

# Create empty DataFrames for each split
train_df = pd.DataFrame()
val_df = pd.DataFrame()
test_df = pd.DataFrame()

# Iterate over classes and splits
for class_name, class_data in files_split.items():
    for split_name, files in class_data.items():
        # Create a DataFrame for the current split
        split_df = pd.DataFrame({'filename': files, 'class': class_name})

        # Append to the corresponding DataFrame
        if split_name == 'train':
            train_df = pd.concat([train_df, split_df], ignore_index=True)
        elif split_name == 'val':
            val_df = pd.concat([val_df, split_df], ignore_index=True)
        elif split_name == 'test':
            test_df = pd.concat([test_df, split_df], ignore_index=True)
# Concatenate all DataFrames into a single frame
dataSet= pd.concat([train_df, val_df, test_df], ignore_index=True)
# Print the DataFrames
print(train_df)
print(val_df)
print(test_df)

                                              filename             class
0    Shop DataSet/non shop lifters/shop_lifter_n_33...  non shop lifters
1    Shop DataSet/non shop lifters/shop_lifter_n_24...  non shop lifters
2    Shop DataSet/non shop lifters/shop_lifter_n_98...  non shop lifters
3    Shop DataSet/non shop lifters/shop_lifter_n_10...  non shop lifters
4    Shop DataSet/non shop lifters/shop_lifter_n_13...  non shop lifters
..                                                 ...               ...
678       Shop DataSet/shop lifters/shop_lifter_95.mp4      shop lifters
679        Shop DataSet/shop lifters/shop_lifter_6.mp4      shop lifters
680  Shop DataSet/shop lifters/videyyyyyyzzzzzyyyss...      shop lifters
681   Shop DataSet/shop lifters/videyyyyyyyyyss_10.mp4      shop lifters
682       Shop DataSet/shop lifters/shop_lifter_67.mp4      shop lifters

[683 rows x 2 columns]
                                             filename             class
0   Shop DataSet/non shop li

In [ ]:
dataSet.head(3) ,len(dataSet)

(                                            filename             class
 0  Shop DataSet/non shop lifters/shop_lifter_n_33...  non shop lifters
 1  Shop DataSet/non shop lifters/shop_lifter_n_24...  non shop lifters
 2  Shop DataSet/non shop lifters/shop_lifter_n_98...  non shop lifters,
 855)

In [ ]:
print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.head(5)

Total videos for training: 683
Total videos for testing: 87


,filename,class
0,Shop DataSet/non shop lifters/shop_lifter_n_33...,non shop lifters
1,Shop DataSet/non shop lifters/shop_lifter_n_24...,non shop lifters
2,Shop DataSet/non shop lifters/shop_lifter_n_98...,non shop lifters
3,Shop DataSet/non shop lifters/shop_lifter_n_10...,non shop lifters
4,Shop DataSet/non shop lifters/shop_lifter_n_13...,non shop lifters


# another sol


--------------------------------------------------------------------------------------------

In [ ]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    #cap.set(cv2.CAP_PROP_POS_MSEC, 20000)
    frames = []
    j = 0
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)
            #cv2.imwrite(data_root+"/train/"+str(j)+".jpg", img)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:

def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [ ]:
import tensorflow as tf
label_processor = tf.keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["class"])
)
print(label_processor.get_vocabulary())

['non shop lifters', 'shop lifters']


Dataset Preprocessing

In [ ]:
def prepare_all_videos(df):
    num_samples = len(df)
    video_paths = df["filename"].values.tolist()
    labels = df["class"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx,path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.hike intern
        #path = video_paths[idx]
        frames = load_video(path)
        frames = frames[None, ...]

        gc.collect()

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_featutes = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            try:
              video_length = batch.shape[1]
              length = min(MAX_SEQ_LENGTH, video_length)
              for j in range(length):
                temp_frame_featutes[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
              temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked
              frame_features[idx,] = temp_frame_featutes.squeeze()
              frame_masks[idx,] = temp_frame_mask.squeeze()
            except:
              #print(i, j, length)
              pass

        gc.collect()
        print(idx)

    return (frame_features, frame_masks), labels

gc.collect()

train_data, train_labels = prepare_all_videos(train_df)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

Frame features in train set: (683, 5, 2048)
Frame masks in train set: (683, 5)


In [ ]:

test_data, test_labels = prepare_all_videos(test_df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86


In [ ]:
input_tensor = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

base = ResNet50(weights="imagenet", include_top=False,input_tensor=input_tensor, input_shape=(IMG_SIZE, IMG_SIZE, 3))
#base.load_weights('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

top = Dense(11, activation='softmax')(
            Dropout(0.5)(
                Dense(2048, activation='relu')(
                    Dropout(0.5)(
                        GlobalAveragePooling2D()(base.output)
                    )
                )
            )
        )

model = Model(input_tensor, top)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:

# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.LSTM(200, return_sequences=True)(
        frame_features_input, mask=mask_input
    )

    x = keras.layers.LSTM(200, return_sequences=True)(x)

    x = keras.layers.GRU(20)(x)
    #x = keras.layers.Dropout(0.4)(x)


    x = keras.layers.Dense(2048, activation="relu")(x)
    x = keras.layers.Dense(1024, activation="relu")(x)

    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(256, activation="relu")(x)

    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(256, activation="relu")(x)

    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


In [ ]:

for layer in model.layers:
    layer.trainable = False

for i in range(-5, 0):
    model.layers[i].trainable = True


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=10e-5),  # Changed 'lr' to 'learning_rate'
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    #filepath = "/tmp/video_classifier"
    #checkpoint = keras.callbacks.ModelCheckpoint(
    #    filepath, save_weights_only=True, save_best_only=True, verbose=1
    #)

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        #callbacks=[checkpoint],
    )

    #seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


history, sequence_model = run_experiment()

Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node functional_2_1/gru_1_1/Assert/Assert defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/lo

---------------------------------------------------------------------------------------------

In [ ]:
df = dataSet.copy()

Set Hyperparameters

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 5
NUM_FEATURES = 2048

# another sol

--------------------------------------------------------------------------------------

In [ ]:
import cv2
import numpy as np



def extract_and_sample_frames(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate frame sampling interval
    sample_interval = max(1, total_frames // num_frames)

    count = 0
    while len(frames) < num_frames and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Sample frames at regular intervals
        if count % sample_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)

        count += 1

    # If fewer frames are extracted, pad with repeated frames
    # Check if frames list is empty before accessing the last element
    while len(frames) < num_frames:
        if frames:  # Check if frames is not empty
            frames.append(frames[-1])
        else:
            # If frames is empty, append a blank frame (or handle the error differently)
            frames.append(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8))
            print(f"Warning: No frames extracted from {video_path}. Appending blank frames.")

    cap.release()
    return frames

In [ ]:
from transformers import VideoMAEFeatureExtractor, VideoMAEForVideoClassification
import torch

# Load pre-trained video transformer
feature_extractor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base")

# Preprocess frames
def preprocess_video_frames(video_path):
    frames = extract_and_sample_frames(video_path, num_frames=16)
    inputs = feature_extractor(frames, return_tensors="pt")
    return inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/377M [00:00<?, ?B/s]

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm  # Import tqdm for progress bar
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import Counter
from sklearn.utils import resample

# Assuming df is your DataFrame with video paths and labels
video_paths = df['filename'].tolist()  # List of video paths
labels = df['class'].tolist()  # List of labels

# Define a mapping from string labels to numeric labels
label_mapping = {
    'shop lifters': 1,
    'non shop lifters': 0
}

numeric_labels = [label_mapping[label] for label in labels]

# Separate the minority and majority classes
minority_class = min(Counter(numeric_labels), key=Counter(numeric_labels).get)
majority_class = max(Counter(numeric_labels), key=Counter(numeric_labels).get)

minority_indices = [i for i, label in enumerate(numeric_labels) if label == minority_class]
majority_indices = [i for i, label in enumerate(numeric_labels) if label == majority_class]

# Oversample the minority class (duplicate minority examples)
minority_oversampled_indices = resample(minority_indices, replace=True, n_samples=len(majority_indices), random_state=42)

# Combine the new dataset indices
balanced_indices = majority_indices + minority_oversampled_indices
balanced_video_paths = [video_paths[i] for i in balanced_indices]
balanced_labels = [numeric_labels[i] for i in balanced_indices]

# Preprocess all videos
inputs_list = []
labels_list = []

# Use tqdm to wrap the zip iterator for progress tracking
for video_path, label in tqdm(zip(balanced_video_paths, balanced_labels), total=len(balanced_video_paths), desc="Processing videos"):
    # Preprocess video frames (this should return a tensor or a dict containing a tensor)
    inputs = preprocess_video_frames(video_path)  # This should return something like {'pixel_values': <tensor>}

    # Assuming 'inputs' is a dict and 'pixel_values' is the key holding the video frames tensor
    if 'pixel_values' in inputs:
        pixel_values = inputs['pixel_values']

        # Ensure that pixel_values is a tensor and has consistent dimensions
        if isinstance(pixel_values, torch.Tensor):
            inputs_list.append(pixel_values)
        else:
            print(f"Error: Expected tensor for pixel values, got {type(pixel_values)}")
            continue  # Skip if the input is invalid
    else:
        print(f"Error: 'pixel_values' not found in inputs for {video_path}")
        continue  # Skip if 'pixel_values' is not found

    # Append the corresponding label
    labels_list.append(label)

# Ensure that there are valid inputs before stacking
if inputs_list:
    # Stack all video tensors into a single tensor
    inputs_tensor = torch.cat(inputs_list, dim=0)  # Concatenate along the batch dimension
    # Convert the labels list to a tensor
    labels_tensor = torch.tensor(labels_list)

    print(f"Inputs tensor shape: {inputs_tensor.shape}")
    print(f"Labels tensor shape: {labels_tensor.shape}")
else:
    print("No valid inputs were processed.")

Processing videos:   0%|          | 0/1062 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:142: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(value)
Processing videos:   0%|          | 1/1062 [00:00<07:18,  2.42it/s]

Processing videos:   0%|          | 2/1062 [00:00<07:29,  2.36it/s]

Processing videos:   0%|          | 3/1062 [00:01<09:23,  1.88it/s]

Processing videos:   0%|          | 4/1062 [00:02<10:28,  1.68it/s]

...

Processing videos: 100%|█████████▉| 1058/1062 [08:06<00:01,  2.53it/s]

Processing videos: 100%|█████████▉| 1059/1062 [08:06<00:01,  2.50it/s]

Processing videos: 100%|█████████▉| 1060/1062 [08:06<00:00,  2.51it/s]

Processing videos: 100%|█████████▉| 1061/1062 [08:07<00:00,  2.51it/s]

Processing videos: 100%|██████████| 1062/1062 [08:07<00:00,  2.18it/s]


--------------------------------------------------------------------------------

Create Helper Functions

In [ ]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
def extract_and_sample_frames(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate frame sampling interval
    sample_interval = max(1, total_frames // num_frames)

    count = 0
    while len(frames) < num_frames and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Sample frames at regular intervals
        if count % sample_interval == 0:
            # Convert to RGB before appending to 'frames'
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)

        count += 1

    cap.release()

    # Check if any frames were extracted
    if not frames:
        print(f"Warning: Video {video_path} has no frames. Skipping...")
        return []  # Return empty list if no frames extracted

    # If fewer frames are extracted, pad with repeated frames
    while len(frames) < num_frames:
        frames.append(frames[-1])

    return frames

In [ ]:
from transformers import VideoMAEFeatureExtractor, VideoMAEForVideoClassification
import torch

# Load pre-trained video transformer
feature_extractor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base")

# Preprocess frames
def preprocess_video_frames(video_path):
    frames = extract_and_sample_frames(video_path, num_frames=16)
    inputs = feature_extractor(frames, return_tensors="pt")
    return inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/377M [00:00<?, ?B/s]

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm  # Import tqdm for progress bar
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import Counter
from sklearn.utils import resample

# Assuming df is your DataFrame with video paths and labels
video_paths = df['filename'].tolist()  # List of video paths
labels = df['class'].tolist()  # List of labels

# Define a mapping from string labels to numeric labels
label_mapping = {
    'shop lifters': 1,
    'non shop lifters': 0
}

numeric_labels = [label_mapping[label] for label in labels]

# Separate the minority and majority classes
minority_class = min(Counter(numeric_labels), key=Counter(numeric_labels).get)
majority_class = max(Counter(numeric_labels), key=Counter(numeric_labels).get)

minority_indices = [i for i, label in enumerate(numeric_labels) if label == minority_class]
majority_indices = [i for i, label in enumerate(numeric_labels) if label == majority_class]

# Oversample the minority class (duplicate minority examples)
minority_oversampled_indices = resample(minority_indices, replace=True, n_samples=len(majority_indices), random_state=42)

# Combine the new dataset indices
balanced_indices = majority_indices + minority_oversampled_indices
balanced_video_paths = [video_paths[i] for i in balanced_indices]
balanced_labels = [numeric_labels[i] for i in balanced_indices]

# Preprocess all videos
inputs_list = []
labels_list = []

# Use tqdm to wrap the zip iterator for progress tracking
for video_path, label in tqdm(zip(balanced_video_paths, balanced_labels), total=len(balanced_video_paths), desc="Processing videos"):
    # Preprocess video frames (this should return a tensor or a dict containing a tensor)
    inputs = preprocess_video_frames(video_path)  # This should return something like {'pixel_values': <tensor>}

    # Assuming 'inputs' is a dict and 'pixel_values' is the key holding the video frames tensor
    if 'pixel_values' in inputs:
        pixel_values = inputs['pixel_values']

        # Ensure that pixel_values is a tensor and has consistent dimensions
        if isinstance(pixel_values, torch.Tensor):
            inputs_list.append(pixel_values)
        else:
            print(f"Error: Expected tensor for pixel values, got {type(pixel_values)}")
            continue  # Skip if the input is invalid
    else:
        print(f"Error: 'pixel_values' not found in inputs for {video_path}")
        continue  # Skip if 'pixel_values' is not found

    # Append the corresponding label
    labels_list.append(label)

# Ensure that there are valid inputs before stacking
if inputs_list:
    # Stack all video tensors into a single tensor
    inputs_tensor = torch.cat(inputs_list, dim=0)  # Concatenate along the batch dimension
    # Convert the labels list to a tensor
    labels_tensor = torch.tensor(labels_list)

    print(f"Inputs tensor shape: {inputs_tensor.shape}")
    print(f"Labels tensor shape: {labels_tensor.shape}")
else:
    print("No valid inputs were processed.")

Processing videos:   0%|          | 0/1062 [00:00<?, ?it/s]

IndexError: list index out of range

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.resnet50.ResNet50(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [ ]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["class"]))

print(label_processor.get_vocabulary())

['non shop lifters', 'shop lifters']


In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2 to filter out INFO and WARNING, 3 to filter out INFO, WARNING, and ERROR

In [ ]:
import sys
import os
import contextlib
import tqdm

@contextlib.contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["filename"].values.tolist()
    labels = df["class"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in tqdm.tqdm(enumerate(video_paths), total=len(video_paths)):
        # Gather all its frames and add a batch dimension.
#         frames = load_video(os.path.join(root_dir, path))
        frames = load_video( path)

        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                with suppress_stdout():

                    temp_frame_features[i, j, :] = feature_extractor.predict(
                        batch[None, j, :]
                    )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

100%|██████████| 87/87 [00:00<00:00, 17108.37it/s]

Frame features in train set: (683, 5, 2048)
Frame masks in train set: (683, 5)


In [ ]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    #filepath = "/tmp/video_classifier"
    #checkpoint = keras.callbacks.ModelCheckpoint(
    #    filepath, save_weights_only=True, save_best_only=True, verbose=1
    #)

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        #callbacks=[checkpoint],
    )

    #seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


history, sequence_model = run_experiment()

Epoch 1/100


InternalError: Graph execution error:

Detected at node functional_4_1/gru_5_1/CudnnRNNV3 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/py

In [ ]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
def to_gif(images):
    converted_images = images.astype(np.uint8)
    imageio.mimsave("animation.gif", converted_images, duration=100)
    return embed.embed_file("animation.gif")

Test Video

In [ ]:
test_video = np.random.choice(test_df["filename"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)


Test video path: Shop DataSet/shop lifters/shop_lifter_40.mp4


NameError: name 'sequence_model' is not defined

In [ ]:
test_frames

NameError: name 'test_frames' is not defined